In [ ]:
# ▓▓  Single-model PDF/TXT summariser  (default: LLaMA-4 Maverick)  ▓▓
# Install once per Colab session:
# !pip install -q --upgrade openai pdfplumber tqdm
!pip install -q --upgrade openai pdfplumber tqdm
import os, io, textwrap, pdfplumber
from google.colab import files
from tqdm.auto import tqdm
from openai import OpenAI, OpenAIError

# ───────────────────────────────────────────────────────────────
# 1️⃣  Interactive settings (one primary model only)
# ───────────────────────────────────────────────────────────────
print("✏️  Configure run — press Enter for values in [brackets]")
api_key  = input("🔑  OpenAI / OpenRouter key : ").strip() or os.getenv("OPENAI_API_KEY")
model    = input("🧠  Model name               [meta-llama/llama-4-maverick] : ").strip() \
           or "meta-llama/llama-4-maverick"
chunk_tk = int(input("✂️  Chunk size (~tokens)    [2000]                       : ") or 2000)
style    = input("🎛️  bullets / paragraph     [bullets]                    : ").strip() or "bullets"
out_name = input("💾  Output file name         [summary]                    : ").strip() or "summary"

if not api_key:
    raise ValueError("❌  API key is required.")

client = OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")

# ───────────────────────────────────────────────────────────────
# 2️⃣  Upload source PDF / TXT
# ───────────────────────────────────────────────────────────────
print("\n📤  Upload a .pdf or .txt file …")
up_file = files.upload()
fname   = next(iter(up_file))
raw     = io.BytesIO(up_file[fname])
ext     = os.path.splitext(fname)[1].lower()

# ───────────────────────────────────────────────────────────────
# 3️⃣  Extract text & chunk
# ───────────────────────────────────────────────────────────────
def pdf_to_text(b: bytes) -> str:
    with pdfplumber.open(io.BytesIO(b)) as pdf:
        return "\n".join((p.extract_text() or "") for p in pdf.pages)

text = pdf_to_text(raw.getvalue()) if ext == ".pdf" else raw.read().decode(errors="ignore")
text = textwrap.dedent(text).replace("\r", "\n").strip()

max_chars = chunk_tk * 4       # ≈ chars per token
chunks = [text[i:i+max_chars] for i in range(0, len(text), max_chars)]
print(f"📚  {len(text):,} chars → {len(chunks)} chunks.\n")

# ───────────────────────────────────────────────────────────────
# 4️⃣  Prompt helper & chat call
# ───────────────────────────────────────────────────────────────
def prompt(txt: str) -> str:
    mode = "bullet-point" if style.lower().startswith("b") else "concise paragraph"
    return (
        f"Provide a {mode} summary of the text below. "
        "Preserve key facts, dates, and names.\n\n"
        f"Text:\n\"\"\"\n{txt}\n\"\"\""
    )

def chat(prompt: str) -> str:
    rsp = client.chat.completions.create(
        model      = model,
        messages   = [{"role": "user", "content": prompt}],
        temperature= 0.2,
        max_tokens = 512,
    )
    return rsp.choices[0].message.content.strip()

# ───────────────────────────────────────────────────────────────
# 5️⃣  Summarise
# ───────────────────────────────────────────────────────────────
summaries = []
for i, chunk in enumerate(tqdm(chunks, desc="🤖  Summarising"), 1):
    try:
        summaries.append(chat(prompt(chunk[:4000])))
    except OpenAIError as e:
        summaries.append(f"[Chunk {i} failed: {e}]")
        print(f"❌  Chunk {i} failed.")

# ───────────────────────────────────────────────────────────────
# 6️⃣  Merge & download
# ───────────────────────────────────────────────────────────────
joiner  = "\n\n" if style.lower().startswith("p") else "\n"
summary = joiner.join(summaries)
outfile = out_name + ".txt"

with open(outfile, "w", encoding="utf-8") as f:
    f.write(summary)

files.download(outfile)
print("🎉  Summary downloaded →", outfile)